<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#package" data-toc-modified-id="package-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>package</a></span></li><li><span><a href="#get-clean-data" data-toc-modified-id="get-clean-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>get clean data</a></span><ul class="toc-item"><li><span><a href="#get-raw-data" data-toc-modified-id="get-raw-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>get raw data</a></span></li><li><span><a href="#test" data-toc-modified-id="test-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>test</a></span></li><li><span><a href="#note" data-toc-modified-id="note-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>note</a></span></li></ul></li><li><span><a href="#tokenize-text-data" data-toc-modified-id="tokenize-text-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>tokenize text data</a></span><ul class="toc-item"><li><span><a href="#prepare-data" data-toc-modified-id="prepare-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>prepare data</a></span></li><li><span><a href="#tokenize-text-with-gensim" data-toc-modified-id="tokenize-text-with-gensim-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>tokenize text with gensim</a></span></li><li><span><a href="#create-corpus_array-&amp;-choose-top-n-word" data-toc-modified-id="create-corpus_array-&amp;-choose-top-n-word-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>create <code>corpus_array</code> &amp; choose top <code>n</code> word</a></span></li></ul></li><li><span><a href="#get-bi_weapon_array" data-toc-modified-id="get-bi_weapon_array-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>get <code>bi_weapon_array</code></a></span></li><li><span><a href="#cross-validation" data-toc-modified-id="cross-validation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>cross validation</a></span><ul class="toc-item"><li><span><a href="#main" data-toc-modified-id="main-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>main</a></span></li><li><span><a href="#base-line" data-toc-modified-id="base-line-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>base line</a></span></li></ul></li><li><span><a href="#test" data-toc-modified-id="test-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>test</a></span></li></ul></div>

## package

In [1]:
## Pre-setting
# automatically adjust the width of the notebook code cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# if one module is changed, this line will automatically reload that module
%load_ext autoreload
%autoreload 2
# display the figure in the notebook
%matplotlib inline
# To change the font size in acrobat
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
## Add path
import os
import sys
src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

In [3]:
import json
import numpy as np
import gensim as gs
from collections import Counter


import pandas as pd
pd.options.display.max_columns = 500

In [4]:
from sklearn.model_selection import train_test_split, KFold


In [5]:
from LLDA.llda import LLDAClassifier

In [6]:
from sklearn.metrics import f1_score, label_ranking_average_precision_score, label_ranking_loss

In [7]:
from evaluation.evaluation import get_label_via_training, doc_class_evaluation_fscore, baseline_doc_class_evaluation_fscore

## get clean data

### get raw data

In [8]:
email_6p2_folder = os.path.abspath(os.path.join(os.pardir, 'data', 's2021_lumen_clean_data'))
email_6p2_file = 's2021_06_20_01_lumen_clean_doc_sia_liwc_classify.csv'
email_6p2_location = os.path.join(email_6p2_folder, email_6p2_file)

In [9]:
email_6p2_df = pd.read_csv(email_6p2_location)

In [10]:
email_6p2_df = email_6p2_df.rename(columns={'scarcity_time':'scarcity'})

### test

In [11]:
print(email_6p2_df.shape)
email_6p2_df.head()

(2771, 35)


,raw_text_id,raw_text,text_type,nostop_stem_doc,nostop_stem_doc_len,clean_doc,clean_doc_len,pos_sia,compound_sia,neu_sia,neg_sia,posemo_liwc,negemo_liwc,anx_liwc,anger_liwc,sad_liwc,reward_liwc,risk_liwc,time_liwc,money_liwc,Authority or Expertise/Source Credibility,Blame/guilt,Commitment,Commitment- Call to Action,Commitment- Indignation,Emphasis,Gain framing,Liking,Loss framing,Objectivity,Reciprocation,Scarcity/Urgency/Opportunity,Social Proof,Social Proof- Admonition,Subjectivity
0,0,"""A Baker Swept By,"" by Edward Hirsch Audio: Re...",news left,baker swept edward hirsch audio read author al...,69,a baker swept by by edward hirsch audio read b...,137,0.075,0.7506,0.905,0.020,4,1,0,0,1,0,1,14,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,1,"""Get Out"" Won’t Have A 100% Rating On Rotten T...",news left,get rate rotten tomato ever fact jordan peel g...,99,get out won t have a rating on rotten tomatoes...,194,0.115,-0.5607,0.772,0.113,6,6,0,0,0,7,1,10,0,1,1,1,0,0,0,0,1,0,1,0,0,0,0,1
2,3,"""Know Your Rights or Your Safety Is At Risk In...",russian ad,know right safeti risk interact polic shock vi...,76,know your rights or your safety is at risk in ...,146,0.068,-0.8751,0.790,0.142,4,2,1,0,0,2,4,3,1,0,1,1,1,1,1,0,0,0,1,0,0,1,1,1
3,4,"""Nancy Pelosi was drunk again today,"" begins a...",fake news,nanci pelosi drunk today begin post recent sha...,90,nancy pelosi was drunk again today begins a po...,172,0.078,-0.1027,0.831,0.092,6,1,0,0,0,5,0,11,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1
4,5,"""Obama out"": POTUS ends speech with viral mic ...",news left,obama potu end speech viral mic drop presid ba...,58,obama out potus ends speech with viral mic dro...,105,0.047,0.1779,0.913,0.040,2,1,0,0,0,1,0,6,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,1


In [12]:
email_6p2_df.columns

Index(['raw_text_id', 'raw_text', 'text_type', 'nostop_stem_doc',
       'nostop_stem_doc_len', 'clean_doc', 'clean_doc_len', 'pos_sia',
       'compound_sia', 'neu_sia', 'neg_sia', 'posemo_liwc', 'negemo_liwc',
       'anx_liwc', 'anger_liwc', 'sad_liwc', 'reward_liwc', 'risk_liwc',
       'time_liwc', 'money_liwc', 'Authority or Expertise/Source Credibility',
       'Blame/guilt', 'Commitment', 'Commitment- Call to Action',
       'Commitment- Indignation', 'Emphasis', 'Gain framing', 'Liking',
       'Loss framing', 'Objectivity', 'Reciprocation',
       'Scarcity/Urgency/Opportunity', 'Social Proof',
       'Social Proof- Admonition', 'Subjectivity'],
      dtype='object')

### note

- there is no short pre-processed doc
- there is 1,763 emails in total

## tokenize text data

### prepare data

In [13]:
doc_list = [i.split() for i in email_6p2_df.nostop_stem_doc]

In [14]:
len(doc_list)

2771

In [15]:
doc_list[:3]

[['baker',
  'swept',
  'edward',
  'hirsch',
  'audio',
  'read',
  'author',
  'alreadi',
  'lose',
  'eyesight',
  'last',
  'winter',
  'rome',
  'paus',
  'doorway',
  'nine',
  'clock',
  'saturday',
  'morn',
  'baker',
  'swept',
  'shini',
  'bicycl',
  'wave',
  'cap',
  'sing',
  'breath',
  'know',
  'baker',
  'wore',
  'white',
  'apron',
  'dust',
  'flour',
  'float',
  'around',
  'citi',
  'like',
  'angel',
  'freshli',
  'bake',
  'day',
  'sure',
  'morn',
  'halt',
  'street',
  'stood',
  'doorway',
  'baker',
  'wing',
  'weekend',
  'morn',
  'new',
  'pristin',
  'look',
  'sky',
  'one',
  'undiminish',
  'instant',
  'misplac',
  'time',
  'saw',
  'bright',
  'bright',
  'everywher',
  'shadow',
  'cross',
  'rooftop',
  'blot'],
 ['get',
  'rate',
  'rotten',
  'tomato',
  'ever',
  'fact',
  'jordan',
  'peel',
  'get',
  'one',
  'film',
  'thriller',
  'sit',
  'impress',
  'fresh',
  'rotten',
  'tomato',
  'one',
  'point',
  'film',
  'held',
  'stea

### tokenize text with gensim

In [16]:
dictionary = gs.corpora.Dictionary(doc_list)

In [17]:
len(dictionary.keys())

14938

In [18]:
corpus = [dictionary.doc2bow(text) for text in doc_list]

In [19]:
dictionary[0]

'alreadi'

In [20]:
len(corpus)

2771

### create `corpus_array` & choose top `n` word

In [21]:
top_n = len(dictionary.keys())
# top_n = 8000

In [22]:
corpus_array = np.zeros([len(corpus), len(dictionary.keys())])

In [23]:
for tmp_doc_id, tmp_doc in enumerate(corpus):
    
    for tmp_word_id, tmpword_cnt in tmp_doc:
        corpus_array[tmp_doc_id][tmp_word_id] = tmpword_cnt

In [24]:

word_cnt_array = corpus_array.sum(axis=0)
top_n_index = word_cnt_array.argsort()[-top_n:][::-1]

In [25]:
word_cnt_array[top_n_index[:5]]

array([1573., 1443., 1071.,  976.,  871.])

In [26]:
corpus_array = corpus_array[:, top_n_index]

In [27]:
corpus_array.shape

(2771, 14938)

## get `bi_weapon_array`

In [28]:
# 2021-06-30
influence_list = [
    'Authority or Expertise/Source Credibility',
    'Commitment',
    'Commitment- Call to Action',
    'Subjectivity',
    'Gain framing',
    'Blame/guilt',
    'Emphasis',
]

influence_list

['Authority or Expertise/Source Credibility',
 'Commitment',
 'Commitment- Call to Action',
 'Subjectivity',
 'Gain framing',
 'Blame/guilt',
 'Emphasis']

In [29]:
bi_weapon_array = email_6p2_df[influence_list].values.astype(int)

In [30]:
print(bi_weapon_array.shape)
bi_weapon_array

(2771, 7)


array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 1, 0],
       [0, 1, 1, ..., 0, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1]])

In [31]:
bi_weapon_array[0]

array([0, 0, 0, 1, 0, 0, 0])

## cross validation

### main

In [32]:
kf = KFold(n_splits=5)

In [33]:
df_result_save = pd.DataFrame()

for train_index, test_index in kf.split(corpus_array):
    # split all data into train & test
    print("TRAIN:", len(train_index), "TEST:", len(test_index))
    corpus_array_train = corpus_array[train_index]
    corpus_array_test = corpus_array[test_index]
    bi_weapon_array_train = bi_weapon_array[train_index]
    bi_weapon_array_test = bi_weapon_array[test_index]
    
    # train the model
    llda = LLDAClassifier(alpha = .05/bi_weapon_array_train.shape[1], threshold=1/bi_weapon_array_train.shape[1])
    llda.fit(corpus_array_train, bi_weapon_array_train)
    
    # predict p(t, d)
    ptd_predict_test = llda.predict_proba(corpus_array_test)
    
    bi_weapon_predict_test = get_label_via_training(ptd_predict_test, bi_weapon_array_train)
    
    # evaluation
    tmp_f1_score_dic = doc_class_evaluation_fscore(bi_weapon_predict_test, bi_weapon_array_test)
    
    # save result
    
    df_result_save = df_result_save.append(tmp_f1_score_dic, ignore_index=True)
    



TRAIN: 2216 TEST: 555


feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix

Number of documents          = 2216
Number of words              = 14938
Number of latent classes     = 7
Number of iteration          = 100
iteration  1/100..
	likelihood ... -1343030.43039804
iteration  2/100..
	likelihood ... -1301902.85738475
iteration  3/100..
	likelihood ... -1272414.29395226
iteration  4/100..
	likelihood ... -1251720.58353045
iteration  5/100..
	likelihood ... -1237719.68772501
iteration  6/100..
	likelihood ... -1228475.11109172
iteration  7/100..
	likelihood ... -1221592.81176962
iteration  8/100..
	likelihood ... -1215693.44471896
iteration  9/100..
	likelihood ... -1211871.19152957
iteration 10/100..
	likelihood ... -1208501.55130901
iteration 11/100..
	likelihood ... -1206191.52269459
iteration 12/100..
	likelihood ... -1203925.84279125
iteration 13/100..
	likelihood ... -1201955.98194620
iteration 14/100..
	likelihood ... -1200200.96283924
iteration 15/100..
	likelihood ... -1199029.31516672
iteration 16/100..
	likelihood ... -1197425.92968136
iteration 1

	word 39 /  39..
document 74 / 555..
	word 73 /  73..
document 75 / 555..
	word 59 /  59..
document 76 / 555..
	word 50 /  50..
document 77 / 555..
	word 74 /  74..
document 78 / 555..
	word 63 /  63..
document 79 / 555..
	word 67 /  67..
document 80 / 555..
	word 70 /  70..
document 81 / 555..
	word 70 /  70..
document 82 / 555..
	word 58 /  58..
document 83 / 555..
	word 77 /  77..
document 84 / 555..
	word 65 /  65..
document 85 / 555..
	word 61 /  61..
document 86 / 555..
	word 84 /  84..
document 87 / 555..
	word 41 /  41..
document 88 / 555..
	word 50 /  50..
document 89 / 555..
	word 23 /  23..
document 90 / 555..
	word 60 /  60..
document 91 / 555..
	word 60 /  60..
document 92 / 555..
	word 79 /  79..
document 93 / 555..
	word 60 /  60..
document 94 / 555..
	word 78 /  78..
document 95 / 555..
	word 41 /  41..
document 96 / 555..
	word 28 /  28..
document 97 / 555..
	word 62 /  62..
document 98 / 555..
	word 45 /  45..
document 99 / 555..
	word 65 /  65..
document 100 / 555..


	word 64 /  64..
document 295 / 555..
	word 44 /  44..
document 296 / 555..
	word 82 /  82..
document 297 / 555..
	word 78 /  78..
document 298 / 555..
	word 14 /  14..
document 299 / 555..
	word 57 /  57..
document 300 / 555..
	word 50 /  50..
document 301 / 555..
	word 28 /  28..
document 302 / 555..
	word 53 /  53..
document 303 / 555..
	word 43 /  43..
document 304 / 555..
	word 67 /  67..
document 305 / 555..
	word 60 /  60..
document 306 / 555..
	word 62 /  62..
document 307 / 555..
	word 80 /  80..
document 308 / 555..
	word 77 /  77..
document 309 / 555..
	word 75 /  75..
document 310 / 555..
	word 71 /  71..
document 311 / 555..
	word 68 /  68..
document 312 / 555..
	word 62 /  62..
document 313 / 555..
	word 71 /  71..
document 314 / 555..
	word 84 /  84..
document 315 / 555..
	word 47 /  47..
document 316 / 555..
	word 47 /  47..
document 317 / 555..
	word 61 /  61..
document 318 / 555..
	word 13 /  13..
document 319 / 555..
	word 46 /  46..
document 320 / 555..
	word 58 /  

...

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix: suspicious line:

feature_matrix

Number of documents          = 2217
Number of words              = 14937
Number of latent classes     = 7
Number of iteration          = 100
iteration  1/100..
	likelihood ... -1363008.28921742
iteration  2/100..
	likelihood ... -1322686.92542709
iteration  3/100..
	likelihood ... -1292711.79286603
iteration  4/100..
	likelihood ... -1273211.10653730
iteration  5/100..
	likelihood ... -1259422.63764317
iteration  6/100..
	likelihood ... -1248758.38085661
iteration  7/100..
	likelihood ... -1240592.16858325
iteration  8/100..
	likelihood ... -1235192.19927958
iteration  9/100..
	likelihood ... -1230829.43073416
iteration 10/100..
	likelihood ... -1227606.25203609
iteration 11/100..
	likelihood ... -1224856.21645117
iteration 12/100..
	likelihood ... -1222122.82743630
iteration 13/100..
	likelihood ... -1219410.87635148
iteration 14/100..
	likelihood ... -1217668.75430428
iteration 15/100..
	likelihood ... -1216014.75114599
iteration 16/100..
	likelihood ... -1214474.24497175
iteration 1

	word 39 /  39..
document 73 / 554..
	word 34 /  34..
document 74 / 554..
	word 47 /  47..
document 75 / 554..
	word 75 /  75..
document 76 / 554..
	word 99 /  99..
document 77 / 554..
	word 51 /  51..
document 78 / 554..
	word 57 /  57..
document 79 / 554..
	word 31 /  31..
document 80 / 554..
	word 100 / 100..
document 81 / 554..
	word 48 /  48..
document 82 / 554..
	word 45 /  45..
document 83 / 554..
	word  7 /   7..
document 84 / 554..
	word 76 /  76..
document 85 / 554..
	word 27 /  27..
document 86 / 554..
	word 16 /  16..
document 87 / 554..
	word 45 /  45..
document 88 / 554..
	word 39 /  39..
document 89 / 554..
	word 42 /  42..
document 90 / 554..
	word 44 /  44..
document 91 / 554..
	word 68 /  68..
document 92 / 554..
	word 79 /  79..
document 93 / 554..
	word 61 /  61..
document 94 / 554..
	word 15 /  15..
document 95 / 554..
	word 54 /  54..
document 96 / 554..
	word 61 /  61..
document 97 / 554..
	word 74 /  74..
document 98 / 554..
	word 55 /  55..
document 99 / 554..


	word 32 /  32..
document 291 / 554..
	word 40 /  40..
document 292 / 554..
	word 39 /  39..
document 293 / 554..
	word 45 /  45..
document 294 / 554..
	word 82 /  82..
document 295 / 554..
	word 46 /  46..
document 296 / 554..
	word 51 /  51..
document 297 / 554..
	word 50 /  50..
document 298 / 554..
	word 60 /  60..
document 299 / 554..
	word 50 /  50..
document 300 / 554..
	word 47 /  47..
document 301 / 554..
	word 58 /  58..
document 302 / 554..
	word 47 /  47..
document 303 / 554..
	word 86 /  86..
document 304 / 554..
	word 86 /  86..
document 305 / 554..
	word 64 /  64..
document 306 / 554..
	word 45 /  45..
document 307 / 554..
	word 56 /  56..
document 308 / 554..
	word 59 /  59..
document 309 / 554..
	word 55 /  55..
document 310 / 554..
	word 23 /  23..
document 311 / 554..
	word 78 /  78..
document 312 / 554..
	word 57 /  57..
document 313 / 554..
	word 58 /  58..
document 314 / 554..
	word 67 /  67..
document 315 / 554..
	word 62 /  62..
document 316 / 554..
	word 54 /  

	word 73 /  73..
document 509 / 554..
	word 30 /  30..
document 510 / 554..
	word 46 /  46..
document 511 / 554..
	word 86 /  86..
document 512 / 554..
	word 29 /  29..
document 513 / 554..
	word 53 /  53..
document 514 / 554..
	word 40 /  40..
document 515 / 554..
	word 34 /  34..
document 516 / 554..
	word 16 /  16..
document 517 / 554..
	word 40 /  40..
document 518 / 554..
	word 24 /  24..
document 519 / 554..
	word 57 /  57..
document 520 / 554..
	word 57 /  57..
document 521 / 554..
	word 52 /  52..
document 522 / 554..
	word 57 /  57..
document 523 / 554..
	word 29 /  29..
document 524 / 554..
	word 36 /  36..
document 525 / 554..
	word 37 /  37..
document 526 / 554..
	word 31 /  31..
document 527 / 554..
	word 14 /  14..
document 528 / 554..
	word 59 /  59..
document 529 / 554..
	word 46 /  46..
document 530 / 554..
	word 47 /  47..
document 531 / 554..
	word 54 /  54..
document 532 / 554..
	word 81 /  81..
document 533 / 554..
	word 49 /  49..
document 534 / 554..
	word 75 /  

In [34]:
df_result_save

,acc_score,f1_score_macro,f1_score_micro
0,0.654054,0.524658,0.610660
1,0.601083,0.514191,0.581326
2,0.656266,0.542288,0.619469
3,0.645436,0.513301,0.603404
4,0.654203,0.523241,0.612987


In [35]:
df_result_save.mean()

acc_score         0.642208
f1_score_macro    0.523536
f1_score_micro    0.605569
dtype: float64

### base line

In [36]:
df_baseline_f1_score = baseline_doc_class_evaluation_fscore(bi_weapon_array_test)

In [37]:
df_baseline_f1_score

,f1_score_macro,f1_score_micro,acc_score,baseline case
0,0.082784,0.201336,0.537648,all email belong to the same class
1,0.435538,0.468027,0.495874,randomly generate label
2,0.491963,0.584247,0.462352,"all 1, then randomly choose one column and set..."
3,0.574747,0.607290,0.436050,set all as 1


In [38]:
df_baseline_f1_score.mean()

f1_score_macro    0.396258
f1_score_micro    0.465225
acc_score         0.482981
dtype: float64

## test

In [39]:
1

1